# Leveraging AsyncIO in inference pipelines

Tempo includes experimental support for `asyncio`, which provides a way to optimise pipelines.
In particular, `asyncio` can be beneficial in scenarios where most of the heavy lifting is done by downstream models and the pipeline just orchestrates calls across these models.
In this case, most of the time within the pipeline will be spent waiting for the requests from downstream models to come back.
`asyncio` will allow us to process other incoming requests during this waiting time.

This example will walk us through the process of setting up an asynchronous pipeline.
As you will see, it's quite similar to the usual synchronous pipelines.

## Prerequisites

This notebooks needs to be run in the `tempo-examples` conda environment defined below. Create from project root folder:

```bash
conda env create --name tempo-examples --file conda/tempo-examples.yaml
```

## Project Structure

In [1]:
!tree -P "*.py"  -I "__init__.py|__pycache__" -L 2

.
├── artifacts
└── src
    ├── constants.py
    ├── data.py
    ├── tempo.py
    └── train.py

2 directories, 4 files


## Train Models

This section is where as a data scientist you do your work of training models and creating artfacts.
For this example, we will train two sklearn and xgboost classification models using the iris dataset.

These models will be used by our inference pipeline.

In [1]:
import logging
from tempo.utils import logger

logger.setLevel(logging.ERROR)
logging.basicConfig(level=logging.ERROR)

In [6]:
# %load src/train.py
import joblib
import os

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from src.data import IrisData
from src.constants import SKLearnFolder, XGBoostFolder


def train_sklearn(data: IrisData):
    logreg = LogisticRegression(C=1e5)
    logreg.fit(data.X, data.y)

    model_path = os.path.join(SKLearnFolder, "model.joblib")
    with open(model_path, "wb") as f:
        joblib.dump(logreg, f)


def train_xgboost(data: IrisData):
    clf = XGBClassifier()
    clf.fit(data.X, data.y)

    model_path = os.path.join(XGBoostFolder, "model.json")
    clf.save_model(model_path)


In [8]:
from src.data import IrisData
from src.train import train_sklearn, train_xgboost

data = IrisData()

train_sklearn(data)
train_xgboost(data)

[14:52:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/agm/.virtualenvs/tempo/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Create Tempo Artifacts

Here we create the Tempo models and orchestration Pipeline for our final service using our models.
For illustration the final service will call the sklearn model and based on the result will decide to return that prediction or call the xgboost model and return that prediction instead.

In [1]:
from src.tempo import classifier

In [5]:
# %load src/tempo.py
import numpy as np

from tempo import ModelFramework, PipelineModels
from tempo.aio import pipeline, Model

from src.constants import ClassifierFolder, SKLearnFolder, XGBoostFolder

SKLearnModel = Model(
    name="test-iris-sklearn",
    platform=ModelFramework.SKLearn,
    local_folder=SKLearnFolder,
    uri="s3://tempo/basic/sklearn",
    description="An SKLearn Iris classification model",
)

XGBoostModel = Model(
    name="test-iris-xgboost",
    platform=ModelFramework.XGBoost,
    local_folder=XGBoostFolder,
    uri="s3://tempo/basic/xgboost",
    description="An XGBoost Iris classification model",
)


@pipeline(
    name="classifier",
    models=PipelineModels(sklearn=SKLearnModel, xgboost=XGBoostModel),
    local_folder=ClassifierFolder,
)
async def classifier(payload: np.ndarray) -> np.ndarray:
    res1 = await classifier.models.sklearn(input=payload)
    if res1[0] > 0.7:
        return res1

    return await classifier.models.xgboost(input=payload)


## Save Classifier Environment

In preparation for running our models we save the Python environment needed for the orchestration to run as defined by a `conda.yaml` in our project.

In [3]:
!cat artifacts/classifier/conda.yaml

name: tempo
channels:
  - defaults
dependencies:
  - python=3.7.9
  - pip:
      - mlops-tempo @ /home/agm/Seldon/tempo
      - mlserver==0.3.1.dev7


In [2]:
import tempo

tempo.save(classifier)

Packing environment at '/home/agm/.conda/envs/tempo-84719eac-03b2-4e0e-be43-b24a8201141d' to '/home/agm/Seldon/tempo/docs/examples/asyncio/artifacts/classifier/environment.tar.gz'
[########################################] | 100% Completed | 10.8s


## Test Locally on Docker

Here we test our models using production images but running locally on Docker. This allows us to ensure the final production deployed model will behave as expected when deployed.

In [3]:
from tempo.seldon.docker import SeldonDockerRuntime

docker_runtime = SeldonDockerRuntime()
docker_runtime.deploy(classifier)
docker_runtime.wait_ready(classifier)

In [6]:
await classifier(np.array([[1, 2, 3, 4]]))

array([2.], dtype=float32)

In [7]:
print(await classifier.remote(np.array([[0, 0, 0,0]])))
print(await classifier.remote(np.array([[5.964,4.006,2.081,1.031]])))

[1.]
[[0.97329617 0.02412145 0.00258233]]


In [8]:
docker_runtime.undeploy(classifier)